Built on Anaconda3 (Python version 3.4) on Windows 10 on 2015-08-10.

This notebook analyzes an Excel file that shows a one-to-many relationship between the text in one column (named 'Company Name' in my work) and the text in another (named 'Ownership Status' in my work). The heart of this work is reversing the direction of the relationship; finding each unique member of the 'many' and showing its relationship to all members in the 'one' population. In effect, it reverses the original one-to-many to a new one-to-many.

I'm sure there is a database term for this, but I am not a database guru. I'm just a guy who needed to find this new relationship from an existing Excel spreadsheet. I always wanted to learn more about Pandas, since I expect to be doing more analysis of spreadsheets in the coming weeks, and I dread the idea of using VBA to do the work.

In [41]:
import pandas as pd

Open the file. Note the use of the leading "r" in the file name, to pass the raw string to Pandas and avoid Unicode issues. The sheetname 'Screening' contains the data I'm using in this analysis.

In [42]:
filename = r'C:\Users\Paul\Documents\03 - Professional\Private Equity\Aerospace and Defense Industry Research\Spaven Analysis\PE_backed_A&D_companies.xlsx'
df = pd.read_excel(filename, sheetname='Screening')

Take a look at the columns in this file, so I can find the owners' names.


In [43]:
df.columns


Index(['Company Name', 'Exchange:Ticker', 'Industry Classifications',
       'Geographic Locations', 'Company Status', 'Business Description',
       'Website', 'Primary Address', 'Ownership Status',
       'Ownership Status_SplitOut', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55

Looking at the Excel, I confirm that the column named 'Ownership Status' contains the "many" side of the relationship that I need to reverse. Now I will ensure the column is a string and explore its structure a bit.

In [53]:
for owners in df['Ownership Status']:
    print(owners.split('; '))

['Atlantic Bridge Capital (Current Sponsor)', 'Audrey Capital (Current Sponsor)', 'Foundry Group (Current Sponsor)', 'J.F. Shea Venture Capital (Current Sponsor)', 'Mayfield Fund (Current Sponsor)', 'Nautilus Ventures LLC (Current Sponsor)', "O'Reilly AlphaTech Ventures, LLC (Current Sponsor)", 'Ooga Labs (Current Sponsor)', 'QUALCOMM Ventures (Current Sponsor)', 'SanDisk Ventures (Current Sponsor)', 'SK Ventures (Current Sponsor)', 'True Ventures (Current Sponsor)', 'WestSummit Capital (Current Sponsor)']
['Lionheart Ventures (Current Sponsor)']
['KKR & Co. L.P. (NYSE:KKR) (Prior Sponsor)']
['Desjardins Business Capital régional et coopératif (Current Sponsor)']
['Fidelity Management & Research Company (Current Sponsor)', 'Franklin Advisory Services, LLC (Current Sponsor)', 'NFJ Investment Group LLC (Current Sponsor)', 'SSgA Funds Management, Inc. (Current Sponsor)', 'Invesco Advisers, Inc. (Prior Sponsor)']
['Argosy Capital (Current Sponsor)', 'Hickory Venture Capital Corp. (Current 

Now that I have found the data of interest, I will convert it, row by row, into a Python dictionary. Keys are the names of owning companies, and values are lists of the names of companies in which they have an ownership interest. This is the 'heart' of this notebook. This is where the reversal of the one-to-many relationship takes place.

In [45]:
owner_to_holdings = {}
for owners in df['Ownership Status']:
    owners_split = owners.split('; ')
    for owner in owners_split:
        if owner not in owner_to_holdings:
            owner_to_holdings[owner] = [df['Company Name']]
        else:
            owner_to_holdings[owner].append(df['Company Name'])


Now a quick look at the structure of the dictionary, checking to see if the one-to-many has been reversed. 

In [46]:
for key in owner_to_holdings.keys():
    print(key + ' has ' + str(len(owner_to_holdings[key])))

Integrated Partners (Current Sponsor) has 1
First Round Capital (Current Sponsor) has 2
Ashford Capital Management Inc. (Prior Sponsor) has 1
RRH Capital Management Inc. (Current Sponsor) has 1
U.S. Venture Partners (Current Sponsor) has 1
Teachers Insurance & Annuity Association of America, Endowment Arm (Prior Sponsor) has 1
Rotunda Capital Partners, LLC (Prior Sponsor) has 2
Presidio STX, LLC (Current Sponsor) has 1
BlackRock Institutional Trust Company, N.A. (Current Sponsor) has 1
H.I.G. Growth Partners, LLC (Prior Sponsor) has 1
Cheyenne Capital Fund, L.P. (Prior Sponsor) has 1
Seed Sumo (Pending Sponsor) has 1
Andlinger & Company, Inc. (Current Sponsor) has 1
Goldman Sachs Australia Capital Markets Limited (Prior Sponsor) has 1
Tencent Holdings Ltd., Investment Arm (Current Sponsor) has 1
Tamarix Capital Corp. (Current Sponsor) has 1
Laurus Capital Management, LLC (Current Sponsor) has 1
Revolution Capital Group, LLC (Prior Sponsor) has 1
Stonehenge Growth Capital, LLC (Prior Sp

Now that I am happy the one-to-many has been reversed, I will convert the dictionary to a list. 

In [47]:
list_of_owners = []
for key, value in owner_to_holdings.items():
    list_of_owners.append([key, len(value)])
    

Now create a Pandas DataFrame with the list of owners and the number of their holdings.

In [48]:
df_owner = pd.DataFrame(list_of_owners)
print(df_owner.sort(columns=1, ascending=False))

                                                      0   1
1020  The Carlyle Group LP (NasdaqGS:CG) (Prior Spon...  14
537               Ranger Aerospace, LLC (Prior Sponsor)  11
135           J.P. Morgan Partners, LLC (Prior Sponsor)   9
819                     Veritas Capital (Prior Sponsor)   8
453              Directional Aviation (Current Sponsor)   8
1058           Acorn Growth Companies (Current Sponsor)   8
19         Advantage Capital Partners (Current Sponsor)   7
1162                Platte River Equity (Prior Sponsor)   7
34       American Industrial Partners (Current Sponsor)   6
844           Fonds de solidarité FTQ (Current Sponsor)   6
665                      Argosy Capital (Prior Sponsor)   6
659   Peninsula Capital Partners, L.L.C. (Current Sp...   5
103   Desjardins Business Capital régional et coopér...   5
757          Onex Corporation (TSX:OCX) (Prior Sponsor)   5
508   The Carlyle Group LP (NasdaqGS:CG) (Current Sp...   5
655                       Golub Capital 

In [52]:
writer = pd.ExcelWriter(r'C:\Users\Paul\Downloads\PandasOutput.xlsx')
df_owner.to_excel(writer, 'PandasOutput')
writer.save()

I am sure there are more elegant and Pythonic ways to do this, but I'm not a programmer, just an analyst happy to add a new tool to my kit--even if I don't know how best to use it yet :)